In [1]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import featuretools as ft
import lightgbm as lgb
from lightgbm import plot_tree
from graphviz import Digraph
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold,GroupKFold, StratifiedKFold
from sklearn.metrics import roc_auc_score,mean_squared_error
import time
import pickle

%matplotlib inline

# Load Data

In [2]:
with open('./data/df_train_total.pickle', 'rb') as handle:
    df_train_total = pickle.load(handle)
    
with open('./data/df_test_total.pickle', 'rb') as handle:
    df_test_total = pickle.load(handle)

In [3]:
df_train_total.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


In [4]:
df_test_total.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,2017-01-01,0,Education,7432,2008.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
1,1,1,0,2017-01-01,0,Education,2720,2004.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
2,2,2,0,2017-01-01,0,Education,5376,1991.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
3,3,3,0,2017-01-01,0,Education,23685,2002.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
4,4,4,0,2017-01-01,0,Education,116607,1975.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6


# Helper Function

In [5]:
def label_encoder(df, categorical_columns=None):
    """Encode categorical values as integers (0,1,2,3...) with pandas.factorize. """
    # if categorical_colunms are not given than treat object as categorical features
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    for col in categorical_columns:
        df[col], uniques = pd.factorize(df[col])
    return df, categorical_columns

In [6]:
df_train_total,colname = label_encoder(df_train_total, categorical_columns=['primary_use'])
df_test_total,colname = label_encoder(df_test_total, categorical_columns=['primary_use'])

# Train with Seperate Meter Type

## General Settings

In [7]:
params = {'objective':'regression',
          'boosting_type':'gbdt',
          'metric':'rmse',
          'learning_rate':0.1,
          'num_leaves': 2**8,
          'max_depth':-1,
          'colsample_bytree':0.5,# feature_fraction 0.7
          'subsample_freq':1,
          'subsample':0.7,
          'verbose':-1,
          'num_threads':8,
          'seed': 47,#42
                } 

In [8]:
category_cols = ['building_id', 'site_id', 'primary_use']
category_cols

['building_id', 'site_id', 'primary_use']

In [9]:
def fold_train_model(splits_num,features_train,labels_train,features_test,categorical):
    splits = splits_num
    #folds = KFold(n_splits = splits,random_state=50)
    folds = StratifiedKFold(n_splits=splits, shuffle=True, random_state=50)
    predictions = np.zeros(len(features_test))
    ave_score = 0
    i = 0
    
    #for fold_num, (trn_idx, val_idx) in enumerate(folds.split(features_train.values, labels_train.values)):
    for fold_num, (trn_idx, val_idx) in enumerate(folds.split(features_train.values, features_train['building_id'])):
    #for trn_idx, val_idx in folds.split(features_train.values, features_train['building_id']):
        print("Fold {}".format(fold_num))
        #print("Fold {}".format(i))
        train_df, y_train_df = features_train.iloc[trn_idx], labels_train.iloc[trn_idx]
        valid_df, y_valid_df = features_train.iloc[val_idx], labels_train.iloc[val_idx]

        trn_data = lgb.Dataset(train_df, label=y_train_df,categorical_feature=categorical)
        val_data = lgb.Dataset(valid_df, label=y_valid_df,categorical_feature=categorical)

        valid_results = {}
        clf = lgb.train(params,
                        trn_data,
                        2000,#10000,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=500,
                        early_stopping_rounds=500,
                        evals_result=valid_results)

        pred = clf.predict(valid_df)
        score = np.sqrt(mean_squared_error(y_valid_df, pred))
        ave_score += score / splits
        predictions += clf.predict(features_test) / splits
    return ave_score,predictions

In [10]:
def train_meter_type(meter_type,df_train_total,df_test_total,category_cols):
    # prepare data
    df_type_train = df_train_total[df_train_total.meter == meter_type]
    # transfer label with log
    df_type_label = np.log1p(df_type_train['meter_reading'])
    df_type_train.drop(columns = ['meter','meter_reading'],inplace=True)
    df_type_train['timestamp'] = df_type_train['timestamp'].astype('int64') // 10**9

    df_type_test = df_test_total[df_test_total.meter == meter_type]
    df_type_row_id = df_type_test['row_id']
    df_type_test.drop(columns = ['row_id','meter'],inplace=True)
    df_type_test['timestamp'] = df_type_test['timestamp'].astype('int64') // 10**9
    
    # train model
    print('train model')
    ave_score,predictions_type = fold_train_model(3,df_type_train,df_type_label,df_type_test,category_cols)
    print('ave socre is %s'%(ave_score))
    
    # get prediction
    print('get prediction')
    sub_type = pd.DataFrame({'row_id': df_type_row_id, 'meter_reading': np.expm1(predictions_type)})
    return sub_type,ave_score

## Train electricity

In [11]:
sub_ele_f,ave_score = train_meter_type(0,df_train_total,df_test_total,category_cols)

train model
Fold 0


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 0.327571	valid_1's rmse: 0.334513
[1000]	training's rmse: 0.298413	valid_1's rmse: 0.310281
[1500]	training's rmse: 0.280634	valid_1's rmse: 0.296729
[2000]	training's rmse: 0.267722	valid_1's rmse: 0.287481
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 0.267722	valid_1's rmse: 0.287481
Fold 1


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 0.327435	valid_1's rmse: 0.335741
[1000]	training's rmse: 0.297968	valid_1's rmse: 0.311142
[1500]	training's rmse: 0.279938	valid_1's rmse: 0.297314
[2000]	training's rmse: 0.267263	valid_1's rmse: 0.288194
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 0.267263	valid_1's rmse: 0.288194
Fold 2


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 0.326485	valid_1's rmse: 0.335758
[1000]	training's rmse: 0.297243	valid_1's rmse: 0.311417
[1500]	training's rmse: 0.279083	valid_1's rmse: 0.297492
[2000]	training's rmse: 0.266501	valid_1's rmse: 0.288561
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 0.266501	valid_1's rmse: 0.288561
ave socre is 0.2880787973504814
get prediction


In [12]:
sub_cw_f,ave_score_cw = train_meter_type(1,df_train_total,df_test_total,category_cols)

train model
Fold 0


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 0.760491	valid_1's rmse: 0.801814
[1000]	training's rmse: 0.680963	valid_1's rmse: 0.749284
[1500]	training's rmse: 0.630262	valid_1's rmse: 0.721143
[2000]	training's rmse: 0.593576	valid_1's rmse: 0.703377
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 0.593576	valid_1's rmse: 0.703377
Fold 1


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 0.75979	valid_1's rmse: 0.799719
[1000]	training's rmse: 0.682722	valid_1's rmse: 0.749662
[1500]	training's rmse: 0.631337	valid_1's rmse: 0.720928
[2000]	training's rmse: 0.594398	valid_1's rmse: 0.703409
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 0.594398	valid_1's rmse: 0.703409
Fold 2


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 0.759525	valid_1's rmse: 0.798168
[1000]	training's rmse: 0.682262	valid_1's rmse: 0.748447
[1500]	training's rmse: 0.631457	valid_1's rmse: 0.720377
[2000]	training's rmse: 0.594616	valid_1's rmse: 0.702201
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 0.594616	valid_1's rmse: 0.702201
ave socre is 0.7029955550635892
get prediction


In [13]:
sub_stm_f,ave_score_stm = train_meter_type(2,df_train_total,df_test_total,category_cols)

train model
Fold 0


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 0.83218	valid_1's rmse: 0.895298
[1000]	training's rmse: 0.766166	valid_1's rmse: 0.872617
[1500]	training's rmse: 0.719067	valid_1's rmse: 0.863097
[2000]	training's rmse: 0.682956	valid_1's rmse: 0.858597
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 0.682956	valid_1's rmse: 0.858597
Fold 1


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 0.833123	valid_1's rmse: 0.892644
[1000]	training's rmse: 0.766584	valid_1's rmse: 0.869346
[1500]	training's rmse: 0.719482	valid_1's rmse: 0.859987
[2000]	training's rmse: 0.683496	valid_1's rmse: 0.856361
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 0.683496	valid_1's rmse: 0.856361
Fold 2


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 0.832302	valid_1's rmse: 0.892071
[1000]	training's rmse: 0.765908	valid_1's rmse: 0.869801
[1500]	training's rmse: 0.718652	valid_1's rmse: 0.861165
[2000]	training's rmse: 0.682688	valid_1's rmse: 0.85733
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 0.682688	valid_1's rmse: 0.85733
ave socre is 0.8574290282577965
get prediction


In [14]:
sub_hw_f,ave_score_hw = train_meter_type(3,df_train_total,df_test_total,category_cols)

train model
Fold 0


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 0.879244	valid_1's rmse: 1.00577
[1000]	training's rmse: 0.780804	valid_1's rmse: 0.986872
[1500]	training's rmse: 0.710717	valid_1's rmse: 0.981584
[2000]	training's rmse: 0.658477	valid_1's rmse: 0.980906
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 0.658477	valid_1's rmse: 0.980906
Fold 1


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 0.880851	valid_1's rmse: 1.00473
[1000]	training's rmse: 0.782428	valid_1's rmse: 0.987715
[1500]	training's rmse: 0.712714	valid_1's rmse: 0.98234
[2000]	training's rmse: 0.660272	valid_1's rmse: 0.980912
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 0.660272	valid_1's rmse: 0.980912
Fold 2


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 500 rounds.
[500]	training's rmse: 0.878137	valid_1's rmse: 1.00666
[1000]	training's rmse: 0.779721	valid_1's rmse: 0.989976
[1500]	training's rmse: 0.709237	valid_1's rmse: 0.985298
[2000]	training's rmse: 0.656501	valid_1's rmse: 0.984156
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 0.656501	valid_1's rmse: 0.984156
ave socre is 0.9819913259489423
get prediction


In [15]:
# StratifiedKFold
(ave_score + ave_score_cw + ave_score_stm + ave_score_hw) / 4

0.7076236766552024

In [61]:
#huber
(ave_score + ave_score_cw + ave_score_stm + ave_score_hw) / 4

1.3604628852421023

In [51]:
# rmse
(ave_score + ave_score_cw + ave_score_stm + ave_score_hw) / 4

1.352893197767166

In [16]:
sub_all = pd.concat([sub_ele_f,sub_cw_f,sub_stm_f,sub_hw_f])
sub_all.sort_values(by='row_id')

,row_id,meter_reading
0,0,163.515640
1,1,63.081669
2,2,5.388535
3,3,137.668903
4,4,998.130488
...,...,...
41697595,41697595,7.466718
41697596,41697596,4.848232
41697597,41697597,0.732145
41697598,41697598,149.104101


In [17]:
sub_all.to_csv('./data/baseline_log_stratified.csv', index = False)